In [10]:
from ultralytics import YOLO

# 1. Define the Dataset Config locally
import yaml
import torch
import os

# Set the correct dataset root path (absolute path as provided)
dataset_root = r'C:/Users/praha/Downloads/JanVision_2.ipynb/janvision_dataset/janvision_dataset'
data_config = {
    'path': dataset_root, # Absolute path to dataset root
    'train': os.path.join(dataset_root, 'images', 'train'),
    'val': os.path.join(dataset_root, 'images', 'val'),
    'names': {
        0: 'Keratoconus',
        1: 'Normal',
        2: 'Suspect'
    }
}

# Save the config file
with open('data.yaml', 'w') as f:
    yaml.dump(data_config, f)

# 2. Train using NVIDIA RTX 3050 (CUDA)
if __name__ == '__main__':
    # Detection for your NVIDIA GPU
    device = '0' if torch.cuda.is_available() else 'cpu'

    if device == '0':
        print(f"✅ Success! RTX 3050 detected. Training on GPU.")
    else:
        print(f"⚠️ GPU not detected. Training on Ryzen CPU. (Check CUDA installation)")

    model = YOLO("yolo11n.pt")  # Load Nano model (Fastest)

    # Train
    model.train(
        data="data.yaml", 
        epochs=15, 
        imgsz=640, 
        batch=16, 
        device=device
    )

⚠️ GPU not detected. Training on Ryzen CPU. (Check CUDA installation)
Ultralytics 8.4.14  Python-3.13.5 torch-2.10.0+cpu CPU (AMD Ryzen 7 7435HS)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, o

## Model Evaluation and Visualization
This section computes the accuracy, F1 score, confusion matrix, ROC curve, and loss curve for your YOLO model. It also compares the accuracy of two classes and checks for overfitting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve, auc, roc_auc_score, precision_score, recall_score
from ultralytics import YOLO
import torch
import os
import glob

# Try to automatically find the best.pt or last.pt file in all subdirectories of runs/detect
weights_path = None
for root, dirs, files in os.walk(r'C:/Users/praha/Downloads/JanVision_2.ipynb/runs/detect'):
    for fname in files:
        if fname in ['best.pt', 'last.pt']:
            weights_path = os.path.join(root, fname)
            break
    if weights_path:
        break

if weights_path and os.path.exists(weights_path):
    print(f'Using weights file: {weights_path}')
    # Load trained model
    model = YOLO(weights_path)

    # Load validation images and labels
    val_images_dir = r'C:/Users/praha/Downloads/JanVision_2.ipynb/janvision_dataset/janvision_dataset/images/val'
    val_labels_dir = r'C:/Users/praha/Downloads/JanVision_2.ipynb/janvision_dataset/janvision_dataset/labels/val'

    # Helper: get image and label paths
    val_image_paths = sorted(glob.glob(os.path.join(val_images_dir, '*.jpg')))
    val_label_paths = sorted(glob.glob(os.path.join(val_labels_dir, '*.txt')))

    # Ensure the number of images and labels match
    if len(val_image_paths) != len(val_label_paths):
        raise ValueError(f"Number of validation images ({len(val_image_paths)}) and labels ({len(val_label_paths)}) do not match. Please check your dataset.")

    # Helper: parse YOLO label file (single class per file assumed)
    def parse_label(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        # Each line: class x_center y_center width height
        return [int(line.split()[0]) for line in lines]

    # Gather ground truth and predictions
    y_true = []
    y_pred = []
    y_scores = []

    for img_path, label_path in zip(val_image_paths, val_label_paths):
        gt_classes = parse_label(label_path)
        y_true.extend(gt_classes)
        
        results = model(img_path)
        pred_classes = [int(box.cls.cpu().numpy()[0]) for box in results[0].boxes] if len(results[0].boxes) > 0 else [-1]
        y_pred.extend(pred_classes)
        # For ROC: get max confidence for each class
        if len(results[0].boxes) > 0:
            y_scores.extend(results[0].boxes.conf.cpu().numpy())
        else:
            y_scores.append(0)

    # Make y_true and y_pred the same length (truncate to shortest)
    min_len = min(len(y_true), len(y_pred))
    y_true = np.array(y_true[:min_len])
    y_pred = np.array(y_pred[:min_len])
    mask = y_pred != -1
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    # Accuracy, F1, Precision, Recall
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    print(f'Validation Accuracy: {acc:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Precision (P): {precision:.4f}')
    print(f'Recall (R): {recall:.4f}')

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6,5))
    plt.imshow(cm, cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.colorbar()
    plt.show()

    # ROC Curve (for each class)
    plt.figure(figsize=(8,6))
    for c in np.unique(y_true):
        y_true_bin = (y_true == c).astype(int)
        y_pred_bin = (y_pred == c).astype(int)
        fpr, tpr, _ = roc_curve(y_true_bin, y_pred_bin)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {c} (AUC = {roc_auc:.2f})')
    plt.plot([0,1],[0,1],'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Validation)')
    plt.legend()
    plt.show()

    # mAP50 (mean Average Precision at IoU=0.5)
    import yaml
    results_path = os.path.join(os.path.dirname(os.path.dirname(weights_path)), 'results.yaml')
    map50 = None
    if os.path.exists(results_path):
        with open(results_path) as f:
            results = yaml.safe_load(f)
        # Print all keys for debugging
        print('results.yaml keys:', list(results.keys()))
        # Print metrics keys if present
        if 'metrics' in results:
            print('metrics keys:', list(results['metrics'].keys()))
        # Try all possible key names for mAP50
        possible_keys = [
            'metrics/mAP_0.5', 'metrics/mAP50', 'metrics/map_50', 'metrics/precision_50',
            'map_50', 'mAP_0.5', 'mAP50', 'precision_50', 'metrics/box/mAP_0.5', 'metrics/box/mAP50'
        ]
        metrics = results.get('metrics', {})
        for k in possible_keys:
            # Try in metrics dict
            if k in metrics:
                map50 = metrics[k]
                break
            # Try in root dict
            if k in results:
                map50 = results[k]
                break
        if map50 is not None:
            print(f'mAP50 (mean Average Precision @ IoU=0.5): {map50:.4f}')
        else:
            print('mAP50 value not found in results.yaml. Please check the printed keys above to locate the correct key.')
        # Loss Curve
        train_loss = None
        val_loss = None
        if 'metrics' in results:
            metrics = results['metrics']
            if 'train/box_loss' in metrics:
                train_loss = metrics['train/box_loss']
            if 'metrics/val/box_loss' in metrics:
                val_loss = metrics['metrics/val/box_loss']
            elif 'val/box_loss' in metrics:
                val_loss = metrics['val/box_loss']
        # Plot loss curves if available
        if train_loss is not None:
            plt.plot(train_loss, label='Box Loss (Train)')
        if val_loss is not None:
            plt.plot(val_loss, label='Box Loss (Val)')
        if train_loss is not None or val_loss is not None:
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Training/Validation Loss Curve')
            plt.legend()
            plt.show()
        # Overfitting check
        if train_loss is not None and val_loss is not None:
            if val_loss[-1] > train_loss[-1]:
                print('Possible overfitting detected: Validation loss is higher than training loss.')

    # Plot accuracy for 2 classes
    if len(np.unique(y_true)) >= 2:
        accs = []
        for c in [0,1]:
            mask = y_true == c
            accs.append(accuracy_score(y_true[mask], y_pred[mask]))
        plt.bar(['Class 0', 'Class 1'], accs)
        plt.ylabel('Accuracy')
        plt.title('Accuracy for Class 0 vs Class 1')
        plt.show()
        print(f'Class 0 Accuracy: {accs[0]:.4f}, Class 1 Accuracy: {accs[1]:.4f}')
        if abs(accs[0] - accs[1]) > 0.1:
            print('Warning: Large accuracy difference between classes.')
else:
    print('Evaluation skipped: No YOLO weights file (best.pt or last.pt) found in any runs/detect subdirectory. Please check your training output.')

Using weights file: C:/Users/praha/Downloads/JanVision_2.ipynb/runs/detect\train7\weights\best.pt

image 1/1 C:\Users\praha\Downloads\JanVision_2.ipynb\janvision_dataset\janvision_dataset\images\val\eye_103_0.jpg: 640x640 1 Suspect, 92.4ms
Speed: 4.3ms preprocess, 92.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\praha\Downloads\JanVision_2.ipynb\janvision_dataset\janvision_dataset\images\val\eye_107_0.jpg: 640x640 1 Keratoconus, 80.9ms
Speed: 3.9ms preprocess, 80.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\praha\Downloads\JanVision_2.ipynb\janvision_dataset\janvision_dataset\images\val\eye_108_0.jpg: 640x640 1 Keratoconus, 81.0ms
Speed: 2.7ms preprocess, 81.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\praha\Downloads\JanVision_2.ipynb\janvision_dataset\janvision_dataset\images\val\eye_114_0.jpg: 640x640 1 Keratoconus, 90.3ms
Speed: 4.1ms preprocess, 90.3ms

<Figure size 600x500 with 2 Axes>

<Figure size 800x600 with 1 Axes>

<Figure size 640x480 with 1 Axes>

Class 0 Accuracy: 0.8929, Class 1 Accuracy: 1.0000
